In [1]:
from google_agent import Google_agent
from deep_research import Deep_research_engine
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field

from langchain_google_genai import ChatGoogleGenerativeAI
import nest_asyncio
nest_asyncio.apply()
from langchain_openai import ChatOpenAI

import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
openai_api_key=os.getenv('openai_api_key')
tavily_client = TavilyClient(api_key=tavily_key)
composio_api_key=os.getenv('composio_api_key')
pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))
GEMINI_MODEL='gemini-2.0-flash'
langchain_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model=GEMINI_MODEL, temperature=0.3)


In [2]:

api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}


In [3]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

@dataclass
class Deps:
    deep_research_output: dict
    google_agent_output: dict

In [4]:
class Cortana_agent:
    def __init__(self, api_keys:dict):
        """
        Args:
            
            api_keys (dict): The API keys to use as a dictionary
        """

        GEMINI_MODEL='gemini-2.0-flash'
        
        pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=api_keys['google_api_key']))
        openai_llm=ChatOpenAI(api_key=api_keys['openai_api_key'])
        langchain_llm = ChatGoogleGenerativeAI(google_api_key=api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3)
        # tools
        llms={'pydantic_llm':pydantic_llm,
              'langchain_llm':langchain_llm,
              'openai_llm':openai_llm}
        deep_research_engine=Deep_research_engine(pydantic_llm,api_keys)
        google_agent=Google_agent(llms,api_keys)
        def google_agent_tool(ctx:RunContext[Deps],query:str):
            """
            Use this tool to interact with the google agent, which can, search for images, manage the user's emails, google tasks, contacts,\
            list its tools and improve its planning if the user gives feedback after a failure.
            This agent can do multiple actions at once so pass the entire query.
            Args:
                query (str): The entire query related to the google agent and its capabilities
            Returns:
                str: The response from the google agent 
            """
            
           
            res=google_agent.chat(query)
            ctx.deps.google_agent_output['node_messages']=res.node_messages
            if google_agent.state.mail_inbox:
                ctx.deps.google_agent_output['inbox']=google_agent.state.mail_inbox
            try:
                return res.node_messages[-1]
            except:
                return res
        


        def search_and_question_answering_tool(ctx: RunContext[Deps], query:str):
            """
            Use this tool to do a deep research on a topic, to gather detailed informations and data, answer_questions from the deep research results or do a quick research if the answer is not related to the deep research.
            Args:
                query (str): The query related to the search_and_question_answering_tool and its capabilities
                

            Returns:
                str: The response from the search_and_question_answering_tool
            """
            @dataclass
            class Route:
                answer: str = Field(default_factory=None,description="the answer to the question if the question is related to the deep research")
                route: str = Field(description="the route, either deep_research or answer_question, or quick_research")
            agent=Agent(pydantic_llm, output_type=Route, instructions="you are a router/question answering agent, you are given a query and you need to decide what to do based on the information provided")
            response= agent.run_sync(f"based on the query: {query}, and the information provided: {ctx.deps.deep_research_output if ctx.deps.deep_research_output else ''} either answer the question or if the answer is not related to the information provided or need more information return 'quick_research' or 'deep_research'")
            route=response.output.route
            if route=='deep_research':
                response=deep_research_engine.chat(query)
                ctx.deps.deep_research_output=response
                return response
            elif route=='answer_question':
                return response.output.answer
            elif route=='quick_research':
                quick_research_agent=Agent(pydantic_llm, tools=[tavily_search_tool(api_keys['tavily_key'])], instructions="do a websearch based on the query")
                result= quick_research_agent.run_sync(query)
                return result.output

        def get_current_time_tool():
            """
            Use this tool to get the current time.
            Returns:
                str: The current time in a formatted string
            """
        
            return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        

        

        @dataclass
        class Cortana_output:
            ui_version: str= Field(description='a markdown format version of the answer for displays if necessary')
            voice_version: str = Field(description='a conversationnal version of the answer for text to voice')
        self.agent=Agent(pydantic_llm, output_type=Cortana_output, tools=[google_agent_tool, search_and_question_answering_tool, get_current_time_tool], system_prompt="you are Cortana, a helpful assistant that can help with a wide range of tasks,\
                          you can use the tools provided to you to help the user with their queries, ask how you can help the user")
        self.memory=Message_state(messages=[])
        self.deps=Deps(deep_research_output={}, google_agent_output={})
    def chat(self, query:str):
        result=self.agent.run_sync(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.output


In [13]:
pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=api_keys['google_api_key']))
openai_llm=ChatOpenAI(api_key=api_keys['openai_api_key'])
langchain_llm = ChatGoogleGenerativeAI(google_api_key=api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3)
# tools
llms={'pydantic_llm':pydantic_llm,
        'langchain_llm':langchain_llm,
        'openai_llm':openai_llm}

In [14]:
goog=Google_agent(llms,api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


In [15]:
res=goog.chat('fetch emails from the inbox')

In [16]:
res.node_messages[-1]

{'mail_manager': {'data': {'messages': [{'attachmentList': [],
     'labelIds': ['UNREAD', 'CATEGORY_UPDATES', 'INBOX'],
     'messageId': '1970ecf96827d3c7',
     'messageText': 'Your job alert for User Experience Designer in North America\r\n29 new jobs match your preferences.\r\n          \r\nUser Interface Designer (UI)\r\nWiraa\r\nUnited States\r\n+170% headcount growth over 2 years\r\nView job: https://www.linkedin.com/comm/jobs/view/4237950707/?trackingId=%2B%2FMv5HmexBcA%2FIJxw04ImA%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Dc623a53c...df2cf745%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3B3ZCBNMhZRJOUTHTfqC4chw%3D%3D&midToken=AQFpLb_YEod9iQ&midSig=1HeJGr1lSwDrM1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-ape4cp~mb5oqkfo~yj-null-null&eid=ape4cp-mb5oqkfo-yj&otpToken=MTQwMTE5ZTMxMzJlYzBjY2IxMjQwNGVkNDIxNmU0YjQ4ZmNmZDc0MDk5YWY4YzYxNzljMDA0NmU0YTUzNThmMmY0ZDVkZmUxMTZjMmUwZjU1MDkzZTBjZDg2NmN

In [9]:
cortana_agent=Cortana_agent(api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use

In [10]:
res=cortana_agent.chat('fetch emails from the inbox')


In [11]:
print(res.ui_version)

I am sorry, I cannot directly display your emails, but I can summarize them for you.


In [24]:
print(res.voice_version)

I am sorry, I am unable to fetch emails. please provide access to your email


In [27]:
cortana_agent.deps.google_agent_output

{'node_messages': []}

In [12]:
cortana_agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are Cortana, a helpful assistant that can help with a wide range of tasks,                          you can use the tools provided to you to help the user with their queries, ask how you can help the user', timestamp=datetime.datetime(2025, 5, 26, 23, 0, 51, 54729, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='fetch emails from the inbox', timestamp=datetime.datetime(2025, 5, 26, 23, 0, 51, 54741, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], instructions=None, kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='google_agent_tool', args={'query': 'fetch emails from the inbox'}, tool_call_id='pyd_ai_5247120ed1f647f89fc2e5b012cd3b46', part_kind='tool-call')], usage=Usage(requests=1, request_tokens=356, response_tokens=11, total_tokens=367, details={}), model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 5, 26, 23, 0, 51, 873070, tzinfo=datetime.tim